<a href="https://colab.research.google.com/github/sdgroeve/ML-course-VIB-2020/blob/master/notebooks/Histone_marks_lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Histone modifications

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

random_seed = 123
np.random.seed(random_seed)

[PyCaret](https://pycaret.org/) is an open source, low-code machine learning library in Python that allows you to go from preparing your data to deploying your model within minutes in your choice of notebook environment.

It is not available by default, so we first install the library.

In [ ]:
! pip install pycaret

# 1. The data

Histone modifications play an important role in affecting gene regulation. Specific histone modifications at specific locations in or near the gene can alter the expression of genes. Predicting gene expression from histone modification signals is a widely studied research topic.

In this competition you will predict gene expression levels (low=0, high=1) based on the presence of histone modifications at specific locations in the gene. You will try to find the model that learns the true underlying model best.

For each gene a region of 10.000bp around the transcription start site of the gene is extracted (5000bp upstream and 5000bp downstream). This region is binned in 100 bins of 100bp. For each bin five core histone modification marks are counted [1].

The dataset is compiled from the "E047" (Primary T CD8+ naive cells from peripheral blood) celltype from Roadmap Epigenomics Mapping Consortium (REMC) database.

[1] Kundaje, A. et al. Integrative analysis of 111 reference human epige-
nomes. Nature, 518, 317–330, 2015.


We start by loading the Pandas library and reading the datasets into Pandas DataFrames:

In [ ]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test.csv")

Let's look at a random sample of the training set:

In [ ]:
train.sample(20)

An alternative visualization for this type of counting data is the heatmap.

In [ ]:
import seaborn as sns

plt.figure(figsize=(30,10))
sns.heatmap(train.sample(20))

The label for each datapoint is in the `Label` column.

In [ ]:
train_labels = train.pop("Label")

Now `train` contains the feature columns only.

Let's look at the number datapoints in each class:

In [ ]:
train_labels.value_counts()

Let's look at `test`:

In [ ]:
test.head(5)

This is a blind test so the `Label` column is not available in the test set. The test set does contain the `GeneId` column that is needed to send your predictions to the Kaggle website.


In [ ]:
test_index_col = test.pop("GeneId")

We can compute some decriptive statistics about the training set using the DataFrame `.describe()` function:

In [ ]:
train.describe()

We can plot these descriptive statistics to get a general overview of the data.

In [ ]:
for col in ["count","mean","std","min","max"]:
  sns.distplot(pd.DataFrame(train.describe().transpose())[col])
  plt.show()

We can use the Pandas `boxplot()` function to plot the feature values.

In [ ]:
plt.figure(figsize=(22,8))
train.boxplot()
plt.show()

Let's plot these for each hisotone mark.

In [ ]:
marks = {}
for m in train.columns:
  marks[m.split("_")[0]] = True
marks = list(marks.keys())
marks

In [ ]:
for mark in marks:
  cols = []
  for m in train.columns:
    if mark in m:
      cols.append(m)
  plt.figure(figsize=(22,8))    
  train[cols].boxplot()
  plt.show()

# 2. Our first model

Let's fit a Logistic Regression model. We will first use the very popular (with good reason) [Scikit-learn](https://scikit-learn.org/stable/) library for that.

First, we hold out 20% if the training data for independant validation.

Next, we fit the modelparameters on the training set and evaluate the fitted model on the validation set.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train,train_labels,
                                                  test_size=.2, random_state=random_seed)

cls = LogisticRegression()

cls.fit(train_X,train_y)

predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)

print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

# 3. How well does it perform in pratice?

In [ ]:
#code for submission
predictions_test = cls.predict(test)

to_write = pd.DataFrame()
to_write["GeneId"] = test_index_col
to_write["Label"] = predictions_test
to_write.to_csv("submission1.csv",index=False)

For the Kaggle competition your predictions are not evaluated by accuracy, but by log-loss:

$$ - \frac{1}{N} \sum_{i=1}^N [y_{i} \log \, p_{i} + (1 - y_{i}) \log \, (1 - p_{i})],$$

where $N$ is the number of datapoints, $y_i$ is the label of datapoint $i$, and $p_i$ is the prediction of the model expressed as a probability.

Let's compute the log-loss:

In [ ]:
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)

print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train),log_loss(val_y,predictions_val)))

# 4. Data pre-processing




Let's scale all the features to [0,1]:

In [ ]:
from sklearn import preprocessing

scaler_minmax = preprocessing.MinMaxScaler()
scaler_minmax.fit(train)
train_norm = pd.DataFrame(scaler_minmax.transform(train),columns=train.columns)
train_norm.head()

In [ ]:
for mark in marks:
  cols = []
  for m in train_norm.columns:
    if mark in m:
      cols.append(m)
  plt.figure(figsize=(22,8))    
  train_norm[cols].boxplot()
  plt.show()

Now, did we improve the predictions?

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                  test_size=.2, random_state=random_seed)

cls.fit(train_X,train_y)

predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)

print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)

print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))

# 5. Hyperparameter tuning

From now on we will use the PyCaret library for fitting models. It provides many useful functions that make Machine Learning fun.

To use PyCaret we first need to setup an environment that prepares the data for fitting and validation.

In [ ]:
from pycaret.classification import *

train_norm["Label"] = train_labels
setup(data=train_norm,target="Label",numeric_features=list(train.columns)[:-1], train_size=0.8, silent=True)

By default, PyCaret does not compute the log-loss metric. So, we add it.

In [ ]:
from sklearn.metrics import log_loss

add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Next, we fit a model using `create_model` that is similar to `fit()` in Scikit-learn, except that it fits and evaluates (CV) on the 80% fitting part of the training set only. The output of this function is a score grid with CV scores by fold.

In [ ]:
lr = create_model("lr")

We just fitted a Logistic Regression model with a default value of 1.0 for the regularization parameter, as you can see below.

In [ ]:
lr

In [ ]:
result = predict_model(lr)

It is now easy to optimize (tune) this hyperparameter.

In [ ]:
tuned_lr = tune_model(lr,tuner_verbose=2, n_iter=20)

This is the 'optimal' value for the regularization hyperparameter.

In [ ]:
tuned_lr

We can now again test this model on the validation set.

In [ ]:
result = predict_model(tuned_lr)

PyCaret offers some very useful tools for model validation.

In [ ]:
evaluate_model(tuned_lr)

Finally, we can fit the optimal model onthe full training set. 

In [ ]:
final_lr = finalize_model(tuned_lr)

And the we can make predictions for the blind test set. PyCaret adds two columns to the test set: Label (the predicted class) and Score (the probability of class 1).

In [ ]:
result = predict_model(final_lr,data=test)
print(result)

In [ ]:
to_write = pd.DataFrame()
to_write["GeneId"] = test_index_col
to_write["Label"] = result["Score"]
to_write.to_csv("submission2.csv",index=False)